# Практическое задание 2 

# Распознавание именованных сущностей из Twitter с помощью LSTM

## курс "Математические методы анализа текстов"


### ФИО: Смирнов Александр Львович

## Введение

### Постановка задачи

В этом задании вы будете использовать рекуррентные нейронные сети для решения проблемы распознавания именованных сущностей (NER). Примерами именованных сущностей являются имена людей, названия организаций, адреса и т.д. В этом задании вы будете работать с данными twitter.

Например, вы хотите извлечь имена и названия организаций. Тогда для текста

    Yan Goodfellow works for Google Brain

модель должна извлечь следующую последовательность:

    B-PER I-PER    O     O   B-ORG  I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система введена, чтобы различать последовательные именованные сущности одного типа.

Решение этого задания будет основано на нейронных сетях, а именно на Bi-Directional Long Short-Term Memory Networks (BiLSTMs). В базовой части задания вам также нужно будет улучшить модель при помощи необучаемого пост-процессинга, основанного на алгоритме Витерби и графической модели CRF. В бонусной части вам будем предложено полноценно использовать связку BiLSTM и CRF, обучая обе модели одновременно.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Numpy](http://www.numpy.org).
 
### Данные

Все данные содержатся в папке `./data`: `./data/train.txt`, `./data/validation.txt`, `./data/test.txt`.

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1s1rFOFMZTBqtJuQDcIvW-8djA78iUDcx)

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

## Часть 1. Подготовка данных (2 балла)

#### Баллы за эту часть можно получить только при успешном выполнении части 2.

### Загрузка данных

Мы будем работать с данными, которые содержат твиты с тегами именованных сущностей. Каждая строка файла содержит пару токен (слово или пунктуация) и тег, разделенные пробелом. Различные твиты разделены пустой строкой.

Функция *read_data* считывает корпус из *file_path* и возвращает два списка: один с токенами и один с соответствующими токенам тегами. Также она заменяет все ники (токены, которые начинаются на символ *@*) на токен `<USR>` и url-ы (токены, которые начинаются на *http://* или *https://*) на токен `<URL>`. 

**<font color='red'>Задание. Реализуйте функцию read_data.</font>**

In [2]:
import re


USERNAME_PATTERN = re.compile("(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)")
LINK_PATTERN = re.compile("https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}")

USERNAME_TOKEN = "<USR>"
LINK_TOKEN = "<URL>"


def read_data(file_path):
    
    tokens, tags = [], []
    current_tokens, current_tags = [], []
    
    with open(file_path) as file:
        for line in file:
            if line == "\n":
                tokens.append(current_tokens)
                tags.append(current_tags)
                current_tokens, current_tags = [], []
            else:
                token, tag = line.split()
                if LINK_PATTERN.search(token):
                    token = LINK_TOKEN
                elif USERNAME_PATTERN.search(token):
                    token = USERNAME_TOKEN
                current_tokens.append(token)
                current_tags.append(tag)
    
    return tokens, tags

Теперь мы можем загрузить 3 части данных:
 - *train* для тренировки модели;
 - *validation* для валидации и подбора гиперпараметров;
 - *test* для финального тестирования.

In [3]:
train_sentences, train_tags = read_data('data/train.txt')
val_sentences, val_tags = read_data('data/validation.txt')
test_sentences, test_tags = read_data('data/test.txt')

Всегда полезно знать, с какими данными вы работаете. Выведем небольшую часть.

In [4]:
for i in range(3):
    for token, one_tag in zip(train_sentences[i], train_tags[i]):
        print('%s\t%s' % (token, one_tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения.

- {token}$\to${token id}: устанавливает соответствие между токеном и строкой в embedding матрице;
- {tag}$\to${tag id}: one hot encoding тегов.


Теперь вам необходимо реализовать функцию *build_dict*, которая должна возвращать словарь {token or tag}$\to${index} и контейнер, задающий обратное отображение.

**<font color='red'>Задание. Реализуйте функцию build_dict.</font>**

In [5]:
def build_dict(entities, special_entities):
    """
    Args:
        entities: a list of lists of tokens or tags
        special_entities: some special tokens
        
    Returns:
        entity_to_idx : mapping to index  
        idx_to_entity : mapping from index
    """
    entity_to_idx = dict()
    idx_to_entity = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0

    idx_to_entity = special_entities + list(set([entity for sentence in entities for entity in sentence]))
    entity_to_idx = dict(zip(idx_to_entity, range(len(idx_to_entity))))
    
    return entity_to_idx, idx_to_entity

После реализации функции *build_dict* вы можете создать словари для токенов и тегов. В нашем случае специальными токенами будут:
 - `<UNK>` токен для обозначаения слов, которых нет в словаре;
 - `<PAD>` токен для дополнения предложений одного батча до одинаковой длины.

In [6]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['<PAD>']

# Create dictionaries 
token_to_idx, idx_to_token = build_dict(train_sentences + val_sentences, special_tokens)
tag_to_idx, idx_to_tag = build_dict(train_tags, special_tags)

### Подготовка датасета и загрузчика

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины. 

Для начала необходимо реализовать <<датасет>> для хранения ваших данных. Датасет должен наследоваться от стандартного pytorch класса `Dataset` и переопределять методы `__getitem__` и `__len__`. Метод `__getitem__` должен возвращать индексированную последовательность и её теги. Не забудьте про `<UNK>` токен для неизвестных слов!

**<font color='red'>Задание. Реализуйте класс TaggingDataset.</font>**

In [7]:
from torch.utils.data import Dataset, DataLoader


class TaggingDataset(Dataset):
    
    def __init__(self, sentences, tags, token_to_idx, tag_to_idx):
        """
        Args:
            sentences: a list of lists of tokens or tags
            tags: some special tokens
            token_to_idx: mapping from token to token indexes
            tag_to_idx: mapping from tag to tag indexes
        """
        super().__init__()
        
        assert len(sentences) == len(tags)
        
        self.sentences_transformed = \
            [torch.tensor(
                 [token_to_idx[token] if token in token_to_idx else token_to_idx["<UNK>"]
                  for token in sentence]) \
             for sentence in sentences]
        
        self.tags_transformed = \
            [torch.tensor(
                 [tag_to_idx[tag] for tag in tag_list]) \
             for tag_list in tags]
        
    def __getitem__(self, idx):
        """
        Args:
            idx : int
            
        Returns:
            sentence_idx : torch.tensor of token indexes
            tag_idx : torch.tensor of tag indexes
        """
        return self.sentences_transformed[idx], self.tags_transformed[idx]
    
    def __len__(self):
        return len(self.sentences_transformed)

Для того, чтобы дополнять последовательности паддингом, будем использовать параметр `collate_fn` класса `DataLoader`. Принимая последовательность пар тензоров для предложений и тегов, необходимо дополнить все последовательности до последовательности максимальной длины в батче. Используйте специальные теги `<PAD>` и `O` для дополнения.

**<font color='red'>Задание. Реализуйте класс PaddingCollator.</font>**

In [8]:
from torch.nn.utils.rnn import pad_sequence


class PaddingCollator:
    def __init__(self,  pad_token_id, pad_tag_id, batch_first=True):
        self.pad_token_idx = pad_token_id
        self.pad_tag_id = pad_tag_id
        self.batch_first = batch_first
        
    def __call__(self, batch):
        """
        Args:
            batch: list of tuples of torch.tensors
        
        Returns:
            new_sentences: torch.tensor
            new_tags: torch.tensor
                Both tensors have the same size 
        """
        sentences = [item[0] for item in batch]
        tags = [item[1] for item in batch]
        
        new_sentences = pad_sequence(sentences, padding_value=self.pad_token_idx, batch_first=self.batch_first)
        new_tags = pad_sequence(tags, padding_value=self.pad_tag_id, batch_first=self.batch_first)
        
        return new_sentences, new_tags

Теперь всё готово, чтобы задать DataLoader. Протестируйте на примере ниже, что всё работает правильно.

In [9]:
small_dataset = TaggingDataset(
    sentences=train_sentences[:7],
    tags=train_tags[:7],
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)

small_loader = DataLoader(
    small_dataset,
    batch_size=3,
    shuffle=False,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['<PAD>'],
        batch_first=True,
    ),
)

batch_lengths = [3, 3, 1]
sequence_lengths = [26, 25, 8]
some_pad_tensor = torch.LongTensor([token_to_idx['<PAD>']] * 12)
some_outside_tensor = torch.LongTensor([[tag_to_idx['<PAD>']] * 12])

for i, (tokens_batch, tags_batch) in enumerate(small_loader):
    assert tokens_batch.dtype == torch.int64, 'tokens_batch is not LongTensor'
    assert tags_batch.dtype == torch.int64, 'tags_batch is not LongTensor'
    
    assert len(tokens_batch) == batch_lengths[i], 'wrong batch length'
    
    for one_token_sequence in tokens_batch:
        assert len(one_token_sequence) == sequence_lengths[i], 'wrong length of sequence in batch'
    
    if i == 0:
        assert torch.all(tokens_batch[2][-12:] == some_pad_tensor), "wrong padding"       
        assert torch.all(tags_batch[2][-12:] == some_outside_tensor), "wrong O tag"

**<font color='red'>Задание. В ячейке ниже задайте датасеты и загрузчики для обучающих, валидационных и тестовых данных.</font>**

In [10]:
BATCH_SIZE = 32
BATCH_FIRST = True
COLLATOR = PaddingCollator(
    pad_token_id=token_to_idx['<PAD>'],
    pad_tag_id=tag_to_idx['<PAD>'],
    batch_first=BATCH_FIRST,
)

train_dataset = TaggingDataset(
    sentences=train_sentences,
    tags=train_tags,
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=COLLATOR
)

val_dataset = TaggingDataset(
    sentences=val_sentences,
    tags=val_tags,
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=COLLATOR
)

test_dataset = TaggingDataset(
    sentences=test_sentences,
    tags=test_tags,
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=COLLATOR
)

## Часть 2. BiLSTM-теггер (4 балла)

Определите архитектуру сети, используя библиотеку pytorch. 

Ваша архитектура в этом пункте должна соответствовать стандартному теггеру (см. лекцию):
* Embedding слой на входе
* Двунаправленный LSTM слой для обработки последовательности
* Используйте dropout (заданный отдельно или встроенный в LSTM) для уменьшения переобучения
* Linear слой на выходе

Для обучения сети используйте поэлементную кросс-энтропийную функцию потерь.
**Обратите внимание**, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь. В качестве оптимизатора рекомендуется использовать Adam. Для получения значений предсказаний по выходам модели используйте функцию $\arg\max$. 

**<font color='red'>Задание. Задайте архитектуру сети и требуемые методы.</font>**

In [11]:
class BiLSTMModel(torch.nn.Module):
    def __init__(
        self,
        vocabulary_size,
        tag_space_size,
        pad_token_idx,
        embedding_dim,
        embeddings_max_norm,
        lstm_hidden_size,
        dropout_zeroed_probability,
        batch_first,
        device='cpu'
    ):
        '''
        Defines neural network structure.
        
        architecture: input -> Embedding -> BiLSTM with Dropout -> Linear
        
        ----------
        Parameters
        
        vocabulary_size: int, number of words in vocabulary.
        tag_space_size: int, number of tags.
        pad_token_idx: int, index of padding character. Used for loss masking.
        embedding_dim: int, dimension of words' embeddings.
        lstm_hidden_size: int, number of hidden units in each LSTM cell
        dropout_zeroed_probability: float, dropout zeroed probability for Dropout layer.
        device: str, cpu or cuda:x
        '''
        super(BiLSTMModel, self).__init__()
        
        
        self.vocabulary_size = vocabulary_size
        self.tag_space_size = tag_space_size
        self.pad_token_idx = pad_token_idx
        self.embedding_dim = embedding_dim
        self.embeddings_max_norm = embeddings_max_norm
        self.lstm_hidden_size = lstm_hidden_size
        self.dropout_zeroed_probability = dropout_zeroed_probability
        self.batch_first = batch_first
        self.device = device
        
        self.embeddings = nn.Embedding(
            num_embeddings=self.vocabulary_size,
            embedding_dim=self.embedding_dim,
            padding_idx=self.pad_token_idx,
            max_norm=self.embeddings_max_norm
        )
        
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.lstm_hidden_size,
            batch_first=self.batch_first,
            bidirectional=True
        )
        
        self.dropout = nn.Dropout(
            p=self.dropout_zeroed_probability
        )
        
        self.linear = nn.Linear(
            in_features=2*self.lstm_hidden_size,
            out_features=self.tag_space_size
        )
        
    def forward(self, x_batch):
        '''
        Makes forward pass.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        x_batch = x_batch.to(self.device)
        embeddings = self.embeddings(x_batch)
        output, (_, _) = self.lstm(embeddings)
        output = self.dropout(output)
        output = self.linear(output)
        return output
    
    def predict_for_batch(self, x_batch):
        '''
        Returns predictions for x_batch. Use argmax function.
        
        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        output = self.forward(x_batch)
        predictions = torch.argmax(output, dim=-1)
        return predictions

Для тестирования сети мы подготовили для вас класс ScoreEvaluator с двумя полезными методами:
 - *predict_tags*: получает батч данных и трансформирует его в список из токенов и предсказанных тегов;
 - *eval_conll*: вычисляет метрики precision, recall и F1

In [12]:
from evaluation_ner import ScoreEvaluator

evaluator = ScoreEvaluator(
    token_to_idx=token_to_idx,
    idx_to_tag=idx_to_tag,
    idx_to_token=idx_to_token,
)

### Эксперименты

Задайте BiLSTMModel. Рекомендуем начать с параметров:
- *batch_size*: 32;
- начальное значение *learning_rate*: 0.01-0.001
- *dropout_zeroed_probability*: 0.5-0.7
- *embedding_dim*: 100-200
- *rnn_hidden_size*: 150-200

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1 мере на валидационной и тестовой выборках было не меньше 0.35. При некотором усердии, можно достичь результата 0.45 по F1 на обоих датасетах.

Если сеть плохо обучается, попробуйте использовать следующие модификации:
    * используйте gradient clipping 
    * ограничивайте норму эмбеддингов через параметр max_norm (сопоставляйте с значениями в клиппинге)
    * на каждой итерации уменьшайте learning rate (например, в 1.1 раз)
    * попробуйте вместо Adam другие оптимизаторы
    * используйте l2 регуляризацию
    * экспериментируйте с значением dropout

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

**<font color='red'>Задание. Проведите требуемые эксперименты.</font>**

In [13]:
import time
import shutil
import os
import torch

from torch.utils.tensorboard import SummaryWriter
    
    
class Trainer:
    
    def __init__(
            self, 
            model, 
            optimizer, 
            criterion, 
            clip,
            lr_scheduler,
            evaluator,
            logdir='./logs', 
            device='cpu'
    ):
        """
            model: объект класса DssmModel
            optimizer: оптимизатор
            criterion: критерий оптимизации
            logdir: директория, в которую SummaryWriter должен писать логи
            device: девайс (cpu или cuda), на котором надо производить вычисления
        """
        self.device = device
        self.model = model.to(self.device)
        self.optimizer = optimizer
        self.criterion = criterion.to(self.device)
        self.clip = clip
        self.lr_scheduler = lr_scheduler
        self.evaluator = evaluator
        self.logdir = logdir
        self._writer = SummaryWriter(log_dir=logdir)
    
    def _calculate_loss(self, batch):
        """
        """
        tokens_batch, tags_batch = batch
        
        tokens_batch = tokens_batch.to(self.device)
        tags_batch = tags_batch.to(self.device)
        
        output = self.model(tokens_batch).permute(0, 2, 1)

        loss = self.criterion(output, tags_batch)
        return loss
    
    def _train_step(self, dataloader):
        """
            dataloader: даталоадер для обучения
            
            returns: лосс на датасете для обучения
        """
        self.model.train()
        epoch_loss = 0.0
        
        for batch in dataloader:
            self.optimizer.zero_grad()
            
            loss = self._calculate_loss(batch)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), self.clip)
            self.optimizer.step()
            
            epoch_loss += loss
        #self.lr_scheduler.step()
            
        return epoch_loss / len(dataloader)
    
    def _eval_step(self, dataloader):
        """
            dataloader: даталоадер для валидации
            
            returns: лосс на валидации
        """
        self.model.eval()
        
        epoch_loss = 0.0
        
        with torch.no_grad():
            for batch in dataloader:
                loss = self._calculate_loss(batch)
                epoch_loss += loss
            metrics = self.evaluator.eval_conll(self.model, dataloader)
            
        return epoch_loss / len(dataloader), metrics["f1"]
    
    def train(self, dataloaders, n_epochs, verbose=True):
        """
            dataloaders: словарь вида {'train': train_dataloader, 'eval': eval_dataloader}
            n_epochs: количество эпох обучения
            verbose: нужно ли выводить каждую эпоху информацию про лоссы
        """
        start = time.time()
        self._n_epoch = 1
        for epoch in range(n_epochs):
            train_loss = self._train_step(dataloaders['train'])
            
            eval_loss, metrics = self._eval_step(dataloaders['eval'])
            if self._writer is not None:
                self._writer.add_scalar('eval/loss', eval_loss, global_step=self._n_epoch)
                
            if verbose:
                print(
                    'epoch: {:>2}, train loss: {:.4f}, eval loss: {:.4f}, eval f1: {:.4f}, time: {:.4f}' \
                        .format(epoch + 1, train_loss, eval_loss, metrics, time.time() - start)
                )
                    
            self._n_epoch += 1

In [14]:
dataloaders = {
    "train": train_loader,
    "eval": val_loader
}

In [15]:
EMBEDDING_DIM = 200
EMBEDDINGS_MAX_NORM = 1
LSTM_HIDDEN_SIZE = 150
DROPOUT_ZEROED_PROBABILITY = 0.7
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)


model = BiLSTMModel(
    vocabulary_size=len(idx_to_token),
    tag_space_size=len(idx_to_tag),
    pad_token_idx=token_to_idx["<PAD>"],
    embedding_dim=EMBEDDING_DIM,
    embeddings_max_norm=EMBEDDINGS_MAX_NORM,
    lstm_hidden_size=LSTM_HIDDEN_SIZE,
    dropout_zeroed_probability=DROPOUT_ZEROED_PROBABILITY,
    batch_first=BATCH_FIRST,
    device=DEVICE
)

cuda


In [16]:
CLIP = 2
LR = 0.001
GAMMA = 0.9
N_EPOCH = 9

criterion = nn.CrossEntropyLoss(ignore_index=tag_to_idx['<PAD>'])
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [17]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    clip=CLIP,
    lr_scheduler=lr_scheduler,
    evaluator=evaluator,
    device=DEVICE
)

In [18]:
trainer.train(dataloaders=dataloaders, n_epochs=N_EPOCH)

epoch:  1, train loss: 0.6373, eval loss: 0.3614, eval f1: 0.0000, time: 2.3380
epoch:  2, train loss: 0.3145, eval loss: 0.3171, eval f1: 9.2166, time: 4.6582
epoch:  3, train loss: 0.2381, eval loss: 0.3149, eval f1: 13.1291, time: 6.9785
epoch:  4, train loss: 0.2075, eval loss: 0.3302, eval f1: 17.0492, time: 9.2879
epoch:  5, train loss: 0.1833, eval loss: 0.3396, eval f1: 20.3947, time: 11.6041
epoch:  6, train loss: 0.1607, eval loss: 0.3539, eval f1: 27.2000, time: 13.9190
epoch:  7, train loss: 0.1303, eval loss: 0.3415, eval f1: 30.3751, time: 16.2291
epoch:  8, train loss: 0.1038, eval loss: 0.3481, eval f1: 31.7164, time: 18.5409
epoch:  9, train loss: 0.0813, eval loss: 0.3484, eval f1: 35.5967, time: 20.8514


In [19]:
trainer._eval_step(test_loader)

(tensor(0.3449, device='cuda:0'), 35.93466424682395)

## Необучаемый пост-процессинг результата (4 балла).

Для обучения нейросетевой модели разметки используется поэлементная кросс-энтропия. При использовании на этапе инференса функции $\arg \max$ для получения выходной последовательности, мы не можем гарантировать согласованность предсказаний. Для согласованности необходимо вместо $\arg \max$ использовать другие функции получения предсказаний.

В модели CRF для получения предсказаний используется алгоритм Витерби. Напомним, что модель CRF моделирует вероятность последовательности $y$ при условии $x$ линейной моделью с вектором весов $w \in \mathbb{R}^d$, которая после некоторых преобразований записывается следующим образом:
$$
p(y|x, w) = \frac{1}{Z(x, w)} \exp\left( \sum_{i=1}^n \sum_{j = 1}^d w_j f_j(y_{i-1}, y_i, x_i, i) \right) =  \frac{1}{Z(x, w)} \exp\left( \sum_{i=1}^n G_{x, i}[y_{i-1}, y_i] \right)
$$

Модель необучаемого пост-процессинга **подробно описана** в приложении к заданию. Она сводится к следующим шагам.

1. Реализовать модель CRF с двумя признаками:
    
    * Лог-софтмакс выходов модели (выход, соответствующий $y_i$ тэгу для i-го токена будем обозначать $S_{i,y_i}$)    
    
    $$
    f_1(y_{i-1}, y_i, x_i, i) = S_{i,y_i}
    $$
    
    * Логарифмы вероятностей переходов

    $$
    f_2(y_{i-1}, y_i, x_i, i) = \log A[v=y_{i}, u=y_{i-1}] \mathbb{I}[i > 1] \times \log C[v = y_i] \mathbb{I}[i = 1], \quad \text{где:}
    $$

    $$A_{vu} = \frac{\sum_{y}\sum_{i=2}^{|y|} \mathbb{I}[y_{i} = v, y_{i - 1} = u]}{\sum_{y}\sum_{i=2}^{|y|} \mathbb{I}[y_{i-1} = u]}
    $$
    $$
    C_v = \frac{\sum_{y}\mathbb{I}[y_{1} = v]}{\sum_{y}1}
    $$
    
2. Реализовать процедуру получения оптимальной выходной последовательности, используя алгоритм Витерби

3. Подобрать на валидационной выборке веса модели $w_1$ и $w_2$

Для исходной модели, дающей на валидационной и тестовой выборке F1 меру 0.408 и 0.46 соответственно, качество после такого пост-процессинга выросло до 0.461 и 0.493. Заметим, что для тестирования модели не нужно переобучать исходную модель. Для более устойчивого поведения модели, используйте сглаживание матрицы $A$ (добавьте перед нормировкой ко всем значениям одинаковое небольшое число).

**<font color='red'>Задание. Реализуйте требуемую модель, добейтесь улучшения качества на валидации и тесте, сделайте выводы.</font>**

In [20]:
class ViterbiPostprocesser:    
    def __init__(self, model, smoothing=1.0, w=1.0):
        """
        model : torch.nn.Module
            Tagging model
        smoothing : float, constant in add-k-smoothing
        w : feature weight
             Use w for first feature weight and (1 - w) for second feature.
        """
        self.model = model
        self.smoothing = smoothing
        self.w = w
        self.log_softmax = nn.LogSoftmax(-1)
        
    def fit(self, dataset):
        """
        Fit the model using maximum likelihood method.
        
        dataset: torch.dataset
            One element if pair (sentence, tags) 
        """
        self.A = torch.zeros((self.model.tag_space_size, self.model.tag_space_size))
        self.C = torch.zeros((self.model.tag_space_size))
        for _, tags in dataset:
            for idx in range(1, len(tags)):
                self.A[tags[idx]][tags[idx - 1]] += 1
            self.C[tags[0]] += 1
            
        self.A += self.smoothing
        self.C += self.smoothing
        
        self.A /= torch.sum(self.A, dim=0).reshape(-1, 1)
        self.C /= len(dataset)
        
        
        self.A = self.A.to(self.model.device)
        self.C = self.C.to(self.model.device)
        
    def decode(self, model_logprobs):
        """
        Viterbi decoding for input model output
        
        model_logprobs : torch.tensor
            Shape is (sequence_length, tag_space_size) 
        """
        B, LENGTH = model_logprobs.shape[0], model_logprobs.shape[1]
        predictions = torch.zeros((B, LENGTH), dtype=torch.long)
        
        for b, sequence in enumerate(model_logprobs):
            final_tokens = torch.zeros((LENGTH), dtype=torch.long)
            for n, distribution in enumerate(sequence):
                if n == 0:
                    final_distribution = self.w * distribution + (1 - self.w) * torch.log(self.C)
                else:
                    final_distribution = self.w * distribution + \
                                        (1 - self.w) * torch.log(self.A[:, final_tokens[n-1].item()])
                
                final_tokens[n] = torch.argmax(final_distribution)
                
            
            predictions[b] = final_tokens
        return predictions
        
    
    def predict_for_batch(self, x_batch):
        """
        Returns predictions for x_batch. Use viterbi decoding.
        
        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        """
        self.model.eval()
        with torch.no_grad():
            out = self.model(x_batch)
        out = self.log_softmax(out)
        
        predictions = self.decode(out)
        
        return predictions

Место для ваших экспериментов:

In [21]:
viterbi_postprocesser = ViterbiPostprocesser(model, w=0.75)
viterbi_postprocesser.fit(train_dataset)

In [22]:
viterbi_postprocesser.model.eval()
with torch.no_grad():
    print(evaluator.eval_conll(viterbi_postprocesser, val_loader))
    print(evaluator.eval_conll(viterbi_postprocesser, test_loader))

{'precision': 49.45054945054945, 'recall': 33.5195530726257, 'f1': 39.955604883462826, 'n_predicted_entities': 364, 'n_true_entities': 537}
{'precision': 48.148148148148145, 'recall': 32.28476821192053, 'f1': 38.65213082259663, 'n_predicted_entities': 405, 'n_true_entities': 604}


## Бонусная часть. Обучаемый постпроцессинг через CRF (2 балла).

Реализуйте сами / модифицируйте открытую реализацию CRF, соответствующую реализации обучаемого пост-процессинга в лекции. Например, можно использовать эту реализацию:
https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

В такой модели должно использоваться два типа признаков:

* $|Y|\times|Y|$ признаков, учитывающих выход модели:
$$	\phi_{uv}(y_{i-1}, y_i, x_i) = \mathbb{I}[y_{i-1} = u]\mathbb{I}[y_i = v] S_{i,y_i}$$
* $|Y|\times|Y|$ признаков, учитывающих связь меток:
$$ \psi_{uv}(y_{i-1}, y_i) = \mathbb{I}[y_{i-1} = u]\mathbb{I}[y_i = v] $$

Итоговое выражение для $G_{x, i}[y_{i-1}, y_i]$ выглядит так:
$$
G_{x, i}[y_{i-1}, y_i] = w(y_{i-1}, y_i) S_{i, y_i} + a(y_{i-1}, y_i)
$$


**<font color='red'>Задание. Обучите модель BiLSTM-CRF в едином пайплайне, добейтесь улучшения качества модели, сделайте выводы по проделанным экспериментам.</font>**

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################        

## Бонусная часть. Дополнительный char-LSTM слой (1 балл).

Добавьте к слою представлений слов обучаемые char-based представления. Каждое слово разделяется на символы и прогоняется через char-based сеть (например, через LSTM). Финальное состояние сети (или конкатенация двух финальных состояний в случае bidirectional LSTM) подаётся как дополнительное представление.

**<font color='red'> Задание. Обучите модель с дополнительным представлением и сравните качество с исходной моделью. Сделайте выводы по проделанным экспериментам.</font>**

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################        